In [1]:
import os
import datetime
from tqdm import tqdm
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from championWrapper import ChampionWrapper

Matplotlib created a temporary config/cache directory at /var/folders/0j/8rftv2kj1s78h279hjwnc1q40000gn/T/matplotlib-9earhwii because the default path (/Users/timinzitari/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
in_data = pd.read_csv('pb_table.csv', index_col=False)
champs = ChampionWrapper()

num_champs = len(champs.champToID)
num_champs


/Users/timinzitari/miniforge3/envs/tensorflow_m1/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


158

In [3]:
#isolate the input frames of hero choices by name
isolated_input = in_data[['BB1', 'RB1', 'BB2', 'RB2', 'BB3', 'RB3', 'BP1', 'RP1','RP2','BP2','BP3', 'RP3', 'RB4','BB4', 'RB5', 'BB5', 'RP4', 'BP4','BP5', 'RP5']]
isolated_input.head()

,BB1,RB1,BB2,RB2,BB3,RB3,BP1,RP1,RP2,BP2,BP3,RP3,RB4,BB4,RB5,BB5,RP4,BP4,BP5,RP5
0,Lulu,Pantheon,Senna,Renekton,Olaf,Camille,Udyr,Twisted Fate,Kai'Sa,Samira,Azir,Taliyah,Rell,Alistar,Gragas,Jayce,Leona,Nautilus,Gangplank,Irelia
1,Lulu,Pantheon,Senna,Renekton,Olaf,Camille,Taliyah,Kai'Sa,Twisted Fate,Samira,Rell,Udyr,Orianna,Alistar,Azir,Nautilus,Leona,Gnar,Syndra,Irelia
2,Renekton,Pantheon,Camille,Lulu,Udyr,Seraphine,Olaf,Thresh,Graves,Senna,Tahm Kench,Aphelios,Twisted Fate,Azir,Karma,Orianna,Syndra,Zoe,Malphite,Gangplank
3,Renekton,Pantheon,Camille,Lulu,Thresh,Twisted Fate,Olaf,Udyr,Kai'Sa,Senna,Tahm Kench,Azir,Orianna,Irelia,Syndra,Jayce,Gnar,Karma,LeBlanc,Blitzcrank
4,Lulu,Pantheon,Lillia,Renekton,Olaf,Rell,Taliyah,Camille,Twisted Fate,Jax,Galio,Udyr,Samira,Senna,Thresh,Alistar,Kai'Sa,Xayah,Leona,Sett


In [4]:
#Dataframe to Numpy
inputnpArray = isolated_input.to_numpy()
inputnpArray


array([['Lulu', 'Pantheon', 'Senna', ..., 'Nautilus', 'Gangplank',
        'Irelia'],
       ['Lulu', 'Pantheon', 'Senna', ..., 'Gnar', 'Syndra', 'Irelia'],
       ['Renekton', 'Pantheon', 'Camille', ..., 'Zoe', 'Malphite',
        'Gangplank'],
       ...,
       ['Renekton', 'Ziggs', 'Thresh', ..., 'Volibear', 'Rakan', 'Braum'],
       ['Ryze', 'LeBlanc', 'Tristana', ..., 'Sejuani', 'Jayce',
        'Cassiopeia'],
       ['Ryze', 'Kennen', 'Tristana', ..., 'Camille', 'Kindred', 'Jayce']],
      dtype=object)

In [5]:
# Numpy convert Names to Ids
for match in range(len(inputnpArray)):
    for column in range(len(inputnpArray[match])):
        inputnpArray[match][column] = champs.cNameToId(inputnpArray[match][column])

inputnpArray



array([[98, 71, 127, ..., 93, 41, 39],
       [98, 71, 127, ..., 114, 107, 39],
       [54, 71, 120, ..., 110, 50, 41],
       ...,
       [54, 97, 139, ..., 90, 145, 121],
       [13, 7, 18, ..., 95, 103, 63],
       [13, 76, 18, ..., 120, 123, 103]], dtype=object)

Here we have the data   
In Raw form we want to make it into [In][Label] sets to    
use in our Sequence to Sequence model   

In [6]:
#Here we have the data
# In Raw form we want to make it into [In][Label] sets to 
# use in our Sequence to Sequence model

In [7]:
numberpd = pd.DataFrame(inputnpArray, columns=['BB1', 'RB1', 'BB2', 'RB2', 'BB3', 'RB3', 'BP1', 'RP1','RP2','BP2','BP3', 'RP3', 'RB4','BB4', 'RB5', 'BB5', 'RP4', 'BP4','BP5', 'RP5'])
print(numberpd)

      BB1 RB1  BB2  RB2  BB3  RB3  BP1  RP1  RP2  BP2  BP3  RP3  RB4  BB4  \
0      98  71  127   54    2  120   68    4  112  138  136  118  149   12   
1      98  71  127   54    2  120  118  112    4  138  149   68   57   12   
2      54  71  120   98   68  113    2  139   88  127  125  151    4  136   
3      54  71  120   98  139    4    2   68  112  127  125  136   57   39   
4      98  71  157   54    2  149  118  120    4   24    3   68  138  127   
...   ...  ..  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
4073    4  13    7   46    5  139   60  126   78  121  151   97  156  103   
4074  139  97   54   60    4   13    7  120  104  103   46  126   92  151   
4075   54  97  139   60  123   39   13    4  120   92   96    5   46  151   
4076   13   7   18  139   92   76   97   39   55  120  144  152   57   75   
4077   13  76   18   41   97   39    7  139  136   92  121   55   46   22   

      RB5  BB5  RP4  BP4  BP5  RP5  
0      70  103   78   93   41   39  
1

In [8]:
traindf, testdf = train_test_split(numberpd, test_size=0.2)
testdf, valdf = train_test_split(testdf, test_size=0.5)

In [9]:
tfl = tf.keras.layers
n_features=num_champs
results = {}
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10)
the_loss = 'mse'
num_epochs=500
from WindowGenerator import SplitGen

splitter = SplitGen()
train, test = train_test_split(inputnpArray, test_size=0.2)
test, val = train_test_split(test, test_size=0.5)

i=18
o=2
f=num_champs

X_train, y_train = splitter.split_sequences(train, i, o, f)
X_test, y_test = splitter.split_sequences(test, i, o, f)
X_val, y_val = splitter.split_sequences(val, i, o, f)


In [10]:
doubleLSTM_model = tf.keras.Sequential([
    tfl.LSTM(512, activation='relu', input_shape=(i, f)),
    tfl.RepeatVector(o),
    tfl.LSTM(256, activation='relu', return_sequences=True),
    tfl.TimeDistributed(tfl.Dense(32)),
    tfl.TimeDistributed(tfl.Dense(f,activation='sigmoid'))
])


doubleLSTM_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

doubleLSTM_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----")
results['doubleLSTM']=doubleLSTM_model.evaluate(X_test, y_test)
print(results['doubleLSTM'])



2022-04-07 04:09:02.285883: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-07 04:09:02.286056: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


102/102 [==============================] - 7s 59ms/step - loss: 0.0857 - accuracy: 0.0066 - val_loss: 0.0063 - val_accuracy: 0.0000e+00


eval-----
13/13 [==============================] - 0s 20ms/step - loss: 0.0063 - accuracy: 0.0000e+00
[0.0063291145488619804, 0.0]


In [11]:
bidoubleLSTM_model = tf.keras.Sequential([
    tfl.Bidirectional(tfl.LSTM(512, activation='relu', input_shape=(i, f))),
    tfl.RepeatVector(o),
    tfl.Bidirectional(tfl.LSTM(256, activation='relu', return_sequences=True)),
    tfl.TimeDistributed(tfl.Dense(64)),
    tfl.TimeDistributed(tfl.Dense(f,activation='sigmoid'))
])
bidoubleLSTM_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

bidoubleLSTM_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----")
results['bidoubleLSTM']=bidoubleLSTM_model.evaluate(X_test, y_test)
print(results['bidoubleLSTM'])

102/102 [==============================] - 11s 97ms/step - loss: 0.0891 - accuracy: 0.0060 - val_loss: 0.0345 - val_accuracy: 0.0000e+00


eval-----
13/13 [==============================] - 0s 32ms/step - loss: 0.0347 - accuracy: 0.0012
[0.03465498983860016, 0.0012254902394488454]


In [12]:
hydoubleLSTM_model = tf.keras.Sequential([
    tfl.Bidirectional(tfl.LSTM(512, activation='relu', input_shape=(i, f))),
    tfl.RepeatVector(o),
    tfl.LSTM(256, activation='relu', return_sequences=True),
    tfl.TimeDistributed(tfl.Dense(64)),
    tfl.TimeDistributed(tfl.Dense(f, activation='sigmoid'))
])
hydoubleLSTM_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

hydoubleLSTM_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----")
results['hydoublelstm']=hydoubleLSTM_model.evaluate(X_test, y_test)
print(results['hydoublelstm'])

102/102 [==============================] - 10s 88ms/step - loss: 0.0590 - accuracy: 0.0046 - val_loss: 0.0219 - val_accuracy: 0.0061


eval-----
13/13 [==============================] - 0s 28ms/step - loss: 0.0221 - accuracy: 0.0025
[0.022066932171583176, 0.0024509804788976908]


In [13]:
lSTM_model = tf.keras.Sequential([
    tfl.LSTM(512, activation='relu', input_shape=(i, f)),
    tfl.RepeatVector(o),
    tfl.TimeDistributed(tfl.Dense(64)),
    tfl.TimeDistributed(tfl.Dense(f))
])
lSTM_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

lSTM_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----")
results['lstm']=lSTM_model.evaluate(X_test, y_test)
print(results['lstm'])

102/102 [==============================] - 5s 50ms/step - loss: 0.0062 - accuracy: 0.0469 - val_loss: 0.0062 - val_accuracy: 0.0539


eval-----
13/13 [==============================] - 0s 16ms/step - loss: 0.0062 - accuracy: 0.0711
[0.006200538482517004, 0.07107843458652496]


In [14]:
bi_LSTM_model = tf.keras.Sequential([
    tfl.Bidirectional(tfl.LSTM(512, activation='relu', input_shape=(i, f))),
    tfl.RepeatVector(o),
    tfl.TimeDistributed(tfl.Dense(64)),
    tfl.TimeDistributed(tfl.Dense(f))
])
bi_LSTM_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

bi_LSTM_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----")
results['bilstm']=bi_LSTM_model.evaluate(X_test, y_test)
print(results['bilstm'])

102/102 [==============================] - 9s 81ms/step - loss: 0.0062 - accuracy: 0.0526 - val_loss: 0.0062 - val_accuracy: 0.0760


eval-----
13/13 [==============================] - 0s 24ms/step - loss: 0.0062 - accuracy: 0.0784
[0.006203063763678074, 0.0784313753247261]


In [15]:
CONV_WIDTH = 3
conv_model = tf.keras.Sequential([

 tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(32, activation='relu', kernel_size=(CONV_WIDTH)),
    tf.keras.layers.Flatten(),
    tfl.Dense(128),
    tf.keras.layers.Dense(o*f,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([o, f])

])
conv_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

conv_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\n\neval-----\n")
results['conv']=conv_model.evaluate(X_test, y_test)
print(results['conv'])

102/102 [==============================] - 0s 1ms/step - loss: 0.0062 - accuracy: 0.0389 - val_loss: 0.0062 - val_accuracy: 0.0502


eval-----

13/13 [==============================] - 0s 506us/step - loss: 0.0062 - accuracy: 0.0539
[0.0062143076211214066, 0.053921569138765335]


In [16]:
clstm_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, activation='relu', kernel_size=(CONV_WIDTH)),
    tfl.MaxPooling1D(pool_size=2),
    tfl.Flatten(),
    tfl.RepeatVector(o),
    tfl.LSTM(512, activation='relu', return_sequences=True), 
    tfl.TimeDistributed(tfl.Dense(256,activation='relu')),
    tfl.TimeDistributed(tfl.Dense(f,activation='sigmoid'))
])

clstm_model.compile(optimizer='adam', loss=the_loss, metrics=['accuracy'])

clstm_model.fit(X_train,y_train, validation_data=(X_val, y_val), callbacks=[es], epochs=num_epochs)
print("\neval-----\n")
results['clstm']=clstm_model.evaluate(X_test, y_test)
print(results['clstm'])

102/102 [==============================] - 1s 10ms/step - loss: 0.0333 - accuracy: 0.0254 - val_loss: 0.0063 - val_accuracy: 0.0294

eval-----

13/13 [==============================] - 0s 3ms/step - loss: 0.0063 - accuracy: 0.0282
[0.006329105701297522, 0.02818627469241619]


In [17]:
print(results)
win = max(results)
print(f'max on test: {win}: {results[win]}')

{'doubleLSTM': [0.0063291145488619804, 0.0], 'bidoubleLSTM': [0.03465498983860016, 0.0012254902394488454], 'hydoublelstm': [0.022066932171583176, 0.0024509804788976908], 'lstm': [0.006200538482517004, 0.07107843458652496], 'bilstm': [0.006203063763678074, 0.0784313753247261], 'conv': [0.0062143076211214066, 0.053921569138765335], 'clstm': [0.006329105701297522, 0.02818627469241619]}
max on test: lstm: [0.006200538482517004, 0.07107843458652496]
